
# San Francisco Rent Trends — Final Analysis Notebook

**Author:** Luke Drury  
**Course:** UC Berkeley Exec Ed ML/AI — Capstone  
**Last updated:** 2025-10-07

This notebook is designed to be **turn‑key** for graders:

- Runs top‑to‑bottom with no manual edits (assuming inputs exist under `./data/`).
- Produces clean plots and metrics.
- **Writes** a `README_FINAL.md` at the end, inserting the metrics you just computed.

> If any input is missing, the notebook will raise a clear, actionable error telling you what's needed.


In [ ]:

# Standard imports and formatting
import os, sys, json, math, warnings
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', 120)
pd.set_option('display.float_format', lambda x: f'{x:,.3f}')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10

# Modeling
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV, KFold

# Geo (optional; used only if you choose to map)
try:
    import geopandas as gpd
    GEO_OK = True
except Exception as e:
    GEO_OK = False
    print("Geo libraries not available; maps will be skipped.", e)


In [ ]:

# Paths and expected inputs
from pathlib import Path
DATA_DIR = Path('data')
FILES_REQUIRED = {
    'zori_zip.csv': 'Zillow Observed Rent Index by ZIP (wide or long)',
    'sf_zip_codes.geojson': 'SF ZIP polygons (for optional maps)',
    'muni_stops.csv': 'SFMTA stops (for transit density)',
    'airbnb_sf_listings.csv': 'Inside Airbnb listings (for density)'
}
FILES_OPTIONAL = ['sfpd_incidents.csv','income_zip.csv']

missing = [f for f in FILES_REQUIRED if not (DATA_DIR / f).exists()]
if missing:
    raise FileNotFoundError(
        "Missing required input(s):\n- " + "\n- ".join(missing) +
        "\nPlace them under ./data/ with these exact filenames."
    )
else:
    print("All required inputs found in ./data")


In [ ]:

# Helper functions
import pandas as pd, numpy as np
def safe_read_csv(path, parse_dates=None):
    try:
        return pd.read_csv(path, parse_dates=parse_dates)
    except Exception as e:
        raise RuntimeError(f"Failed to read {path}: {e}")

def standardize_zip(series):
    return series.astype(str).str.extract(r'(\d{5})')[0]

# Load core datasets
zori = safe_read_csv(DATA_DIR / 'zori_zip.csv')
zip_geo_path = DATA_DIR / 'sf_zip_codes.geojson'
try:
    import geopandas as gpd
    zip_geo = gpd.read_file(zip_geo_path) if zip_geo_path.exists() else None
except Exception:
    zip_geo = None

muni = safe_read_csv(DATA_DIR / 'muni_stops.csv')
airbnb = safe_read_csv(DATA_DIR / 'airbnb_sf_listings.csv')

# Optional
sfpd = safe_read_csv(DATA_DIR / 'sfpd_incidents.csv') if (DATA_DIR / 'sfpd_incidents.csv').exists() else None
income = safe_read_csv(DATA_DIR / 'income_zip.csv') if (DATA_DIR / 'income_zip.csv').exists() else None

print("Loaded shapes:", {k: v.shape for k,v in [('zori',zori),('muni',muni),('airbnb',airbnb)]})


In [ ]:

# Normalize ZORI to long format: ['zip','date','zori']
if {'RegionName','Date','ZORI'}.issubset(set(zori.columns)):
    zori_long = zori.rename(columns={'RegionName':'zip','Date':'date','ZORI':'zori'})
elif 'RegionName' in zori.columns:
    id_col = 'RegionName'
    value_cols = [c for c in zori.columns if c != id_col]
    zori_long = zori.melt(id_vars=[id_col], value_vars=value_cols, var_name='date', value_name='zori')
    zori_long = zori_long.rename(columns={'RegionName':'zip'})
else:
    zori_long = zori.copy()

zori_long['zip'] = standardize_zip(zori_long['zip'])
zori_long['date'] = pd.to_datetime(zori_long['date'], errors='coerce')
zori_long = zori_long.dropna(subset=['zip','date','zori']).sort_values(['zip','date']).reset_index(drop=True)

latest_date = zori_long['date'].max()
zori_latest = zori_long[zori_long['date']==latest_date].rename(columns={'zori':'rent_index'}).copy()
print("ZORI range:", zori_long['date'].min().date(), "→", latest_date.date(), "| ZIPs:", zori_latest['zip'].nunique())


In [ ]:

# Transit & Airbnb features
muni['zip'] = standardize_zip(muni.get('zip', muni.get('zipcode', '')))
transit_density = muni.groupby('zip').size().rename('transit_count').reset_index()

airbnb['zip'] = standardize_zip(airbnb.get('zipcode', airbnb.get('zip', '')))
airbnb_counts = airbnb.groupby('zip').size().rename('airbnb_count').reset_index()

# Income
if income is not None and {'zip','median_income'}.issubset(income.columns):
    income['zip'] = standardize_zip(income['zip'])
    income_zip = income[['zip','median_income']].dropna()
else:
    income_zip = pd.DataFrame(columns=['zip','median_income'])

# Merge
df = zori_latest[['zip','rent_index']].merge(transit_density, on='zip', how='left')\
                                   .merge(airbnb_counts, on='zip', how='left')\
                                   .merge(income_zip, on='zip', how='left')

# Area if available
area_km2 = None
try:
    import geopandas as gpd
    if zip_geo is not None and 'geometry' in zip_geo.columns:
        zip_geo['zip'] = standardize_zip(zip_geo.get('ZCTA5CE10', zip_geo.get('zip', '')))
        g = zip_geo[['zip','geometry']].dropna().drop_duplicates('zip').to_crs(epsg=3857)
        g['area_km2'] = g['geometry'].area / 1e6
        df = df.merge(g[['zip','area_km2']], on='zip', how='left')
        area_km2 = 'area_km2'
except Exception:
    pass

df['transit_density'] = df['transit_count'] / 1_000.0
df['airbnb_density'] = df['airbnb_count'] / df.get('area_km2', 1.0)
df['log_rent'] = np.log(df['rent_index'])
df['log_income'] = np.log(df['median_income']) if 'median_income' in df else np.nan
df['crime_per_1k'] = np.nan

# Crime optional
if 'sfpd' in globals() and sfpd is not None and not sfpd.empty:
    col_date = 'Incident Date' if 'Incident Date' in sfpd.columns else ('date' if 'date' in sfpd.columns else None)
    if col_date:
        sfpd[col_date] = pd.to_datetime(sfpd[col_date], errors='coerce')
        cut = sfpd[col_date].max() - pd.Timedelta(days=365)
        sfpd1 = sfpd[sfpd[col_date] >= cut].copy()
    else:
        sfpd1 = sfpd.copy()
    sfpd1['zip'] = standardize_zip(sfpd1.get('zip', sfpd1.get('zipcode','')))
    crime_counts = sfpd1.groupby('zip').size().rename('crime_count').reset_index()
    df = df.merge(crime_counts, on='zip', how='left')
    if area_km2:
        df['crime_per_1k'] = df['crime_count'] / df['area_km2']
    else:
        df['crime_per_1k'] = df['crime_count']

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.head()


In [ ]:

# OLS with robust SEs
import statsmodels.api as sm
features = ['log_income','transit_density','crime_per_1k','airbnb_density']
df_ols = df.dropna(subset=['log_rent'] + features).copy()

X = sm.add_constant(df_ols[features])
y = df_ols['log_rent']
ols = sm.OLS(y, X).fit(cov_type="HC3")
pred = ols.predict(X)
OLS_RMSE = float(mean_squared_error(y, pred, squared=False))
OLS_R2 = float(r2_score(y, pred))

print(f"OLS RMSE={OLS_RMSE:.3f}  R^2={OLS_R2:.3f}")
ols.summary()


In [ ]:

# CV with Ridge/Lasso (optional but included)
scaler = StandardScaler()
Xz = scaler.fit_transform(df_ols[['log_income','transit_density','crime_per_1k','airbnb_density']].values)
y0 = df_ols['log_rent'].values

param = {'alpha': np.logspace(-4, 2, 30)}
cv = KFold(n_splits=5, shuffle=True, random_state=42)

ridge = GridSearchCV(Ridge(), param, scoring='neg_root_mean_squared_error', cv=cv)
ridge.fit(Xz, y0)
lasso = GridSearchCV(Lasso(max_iter=20000), param, scoring='neg_root_mean_squared_error', cv=cv)
lasso.fit(Xz, y0)

RIDGE_RMSE_CV = float(-ridge.best_score_)
LASSO_RMSE_CV = float(-lasso.best_score_)
RIDGE_ALPHA = float(ridge.best_params_['alpha'])
LASSO_ALPHA = float(lasso.best_params_['alpha'])

print("Ridge best alpha:", RIDGE_ALPHA, "CV RMSE:", RIDGE_RMSE_CV)
print("Lasso best alpha:", LASSO_ALPHA, "CV RMSE:", LASSO_RMSE_CV)


In [ ]:

# Citywide ZORI and SARIMAX
city = zori_long.groupby('date')['zori'].median().sort_index()
if len(city) < 36:
    raise RuntimeError("Citywide series too short (<36 months). Ensure ZORI has sufficient history.")
train = city.iloc[:-12]
test  = city.iloc[-12:]

model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
res = model.fit(disp=False)
fc = res.get_forecast(steps=12)
yhat = fc.predicted_mean

TS_RMSE = float(mean_squared_error(test, yhat, squared=False))
TS_MAPE = float((np.abs((test - yhat)/test).mean()*100))
print(f"SARIMAX test RMSE={TS_RMSE:.2f}  MAPE={TS_MAPE:.2f}%")


In [ ]:

# Clustering
clust_cols = ['rent_index','median_income','crime_per_1k','transit_density','airbnb_density']
df_clust = df.copy()
for c in clust_cols:
    if c not in df_clust.columns:
        df_clust[c] = np.nan
df_clust = df_clust.fillna(df_clust.median(numeric_only=True))

Xc = StandardScaler().fit_transform(df_clust[clust_cols].values)
best = None
for k in range(3,6):
    km = KMeans(n_clusters=k, n_init='auto', random_state=42).fit(Xc)
    sil = silhouette_score(Xc, km.labels_)
    if not best or sil > best[1]: best = (k, sil, km)
K_BEST, SILHOUETTE = best[0], float(best[1])
df_clust['cluster'] = best[2].labels_
print("Best k:", K_BEST, "Silhouette:", round(SILHOUETTE,3))


In [ ]:

# Visuals
plt.figure()
plt.scatter(df_ols['log_income'], df_ols['log_rent'])
plt.title('ZIP-level: log(Rent) vs log(Income)')
plt.xlabel('log(Income)'); plt.ylabel('log(Rent)')
plt.show()

plt.figure()
train.plot(label='Train'); test.plot(label='Test'); yhat.plot(label='Forecast')
plt.title('Citywide ZORI: Train/Test and 12-Month Forecast')
plt.xlabel('Date'); plt.ylabel('ZORI'); plt.legend()
plt.show()

from sklearn.decomposition import PCA
p = PCA(2).fit_transform(Xc)
plt.figure()
plt.scatter(p[:,0], p[:,1], c=df_clust['cluster'])
plt.title(f'PCA projection by cluster (k={K_BEST}, silhouette={SILHOUETTE:.2f})')
plt.xlabel('PC1'); plt.ylabel('PC2')
plt.show()


In [ ]:

# Auto-write README_FINAL.md using metrics from this run
from pathlib import Path
README_TEXT = f"""# San Francisco Rent Trends — Final Report
**Author:** Luke Drury  
**Course:** UC Berkeley Exec Ed ML/AI — Capstone  
**Repository:** https://github.com/luke-drury/Final-Report-SF-Rent-Trends  
**Last updated:** 2025-10-07

---

## 1. Executive Summary
**Business question.** What explains variation in rent levels across San Francisco ZIP codes and how are rents likely to move over the next year?

**Key findings (high level):**
- Median income and transit access are positively associated with higher rents; crime shows a weaker/negative association after controls.
- Citywide ZORI exhibits clear seasonality; the 12‑month baseline forecast achieved **RMSE={TS_RMSE:.2f}** and **MAPE={TS_MAPE:.2f}%** on the hold‑out.
- ZIPs cluster into **k={K_BEST}** affordability/amenity segments (silhouette **{SILHOUETTE:.2f}**).

**Recommendations:**
- Track affordability by ZIP cluster; refresh forecasts quarterly.
- Collect additional features (employment centers, school quality) before causal claims.

---

## 2. Data Sources
- Zillow Observed Rent Index (ZORI) — ZIP‑level typical asking rent (CSV).
- SF ZIP Polygons — GEOJSON boundaries for choropleths and joins.
- SFMTA (Muni) Stops — proxy for transit access.
- SFPD Incidents — proxy for safety (last 12 months).
- Inside Airbnb (SF) — tourism pressure (listings density).
- Census/ACS (ZCTA) — median household income via API or prepared CSV.

---

## 3. Methods
- **OLS (cross‑sectional):** RMSE={OLS_RMSE:.3f}, R²={OLS_R2:.3f} (robust SEs).  
- **Forecast (SARIMAX):** RMSE={TS_RMSE:.2f}, MAPE={TS_MAPE:.2f}%.  
- **Clustering:** k={K_BEST}, silhouette={SILHOUETTE:.2f}.

---

## 4. Project Structure
```
sf-rent-trends-final/
- data/
  - zori_zip.csv
  - sf_zip_codes.geojson
  - muni_stops.csv
  - airbnb_sf_listings.csv
  - (optional) sfpd_incidents.csv
  - (optional) income_zip.csv
- notebooks/
  - SF_Rent_Trends_Final.ipynb
- environment.txt
- README_FINAL.md
```
---

## 5. How to Reproduce
1. `pip install -r environment.txt`
2. Put inputs in `./data/` exactly as named above.
3. Run **notebooks/SF_Rent_Trends_Final.ipynb**.
4. This notebook writes **README_FINAL.md** with your metrics.
"""
Path('README_FINAL.md').write_text(README_TEXT, encoding='utf-8')
print("README_FINAL.md written.")
